## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-25-03-43-43 +0000,nyt,Were You Assigned Full Books to Read in High S...,https://www.nytimes.com/2025/09/24/us/educatio...
1,2025-09-25-03-40-06 +0000,nyt,Giant Brand Bikes Made in Taiwan Barred by Tru...,https://www.nytimes.com/2025/09/24/business/ec...
2,2025-09-25-03-28-44 +0000,startribune,"Two Texas men arrested, charged in home invasi...",https://www.startribune.com/two-texas-men-arre...
3,2025-09-25-03-25-05 +0000,nyt,"With His Pivot on Ukraine, Trump May Be Washin...",https://www.nytimes.com/2025/09/24/us/politics...
4,2025-09-25-03-21-21 +0000,nypost,Trump says UN escalator outage was ‘absolutely...,https://nypost.com/2025/09/24/us-news/trump-sa...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,59
150,ice,16
205,kimmel,16
268,new,16
129,china,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
234,2025-09-24-14-05-20 +0000,nypost,Trump threatens to sue ABC over Jimmy Kimmel’s...,https://nypost.com/2025/09/24/us-news/trump-th...,114
325,2025-09-24-05-04-00 +0000,wsj,Kimmel Makes Emotional Return to ABC With Trum...,https://www.wsj.com/business/media/jimmy-kimme...,112
298,2025-09-24-09-30-00 +0000,wsj,Stephen Miran set out his case for lower inter...,https://www.wsj.com/economy/central-banking/st...,109
112,2025-09-24-21-39-41 +0000,wapo,President Donald Trump wrote on TruthSocial th...,https://www.washingtonpost.com,105
79,2025-09-24-22-55-47 +0000,nypost,Trump’s pro-Ukraine shift a ‘strategic move’ b...,https://nypost.com/2025/09/24/world-news/trump...,102


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
234,114,2025-09-24-14-05-20 +0000,nypost,Trump threatens to sue ABC over Jimmy Kimmel’s...,https://nypost.com/2025/09/24/us-news/trump-th...
96,77,2025-09-24-22-20-12 +0000,nypost,"Vance tears into Democrats, calls out Gavin Ne...",https://nypost.com/2025/09/24/us-news/vance-te...
318,56,2025-09-24-07-04-00 +0000,wsj,China said it will no longer seek new special ...,https://www.wsj.com/economy/trade/china-to-sto...
10,45,2025-09-25-02-51-33 +0000,nypost,"Mom who lost daughter, 15, to suicide after on...",https://nypost.com/2025/09/24/world-news/mom-w...
324,42,2025-09-24-05-11-13 +0000,nypost,Typhoon Ragasa batters Hong Kong and southern ...,https://nypost.com/2025/09/24/world-news/typho...
298,42,2025-09-24-09-30-00 +0000,wsj,Stephen Miran set out his case for lower inter...,https://www.wsj.com/economy/central-banking/st...
223,40,2025-09-24-15-06-31 +0000,bbc,Israeli forces near Gaza City centre as Hamas ...,https://www.bbc.com/news/articles/c4gk9dreldxo...
27,37,2025-09-25-01-34-30 +0000,nypost,Trans boyfriend of Charlie Kirk suspect Tyler ...,https://nypost.com/2025/09/24/us-news/trans-bo...
64,34,2025-09-24-23-34-56 +0000,nypost,NYC GOP candidate Curtis Sliwa drops Guardian ...,https://nypost.com/2025/09/24/us-news/nyc-gop-...
11,33,2025-09-25-02-51-00 +0000,wsj,The Office of Management and Budget told feder...,https://www.wsj.com/politics/policy/white-hous...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
